In [1]:
import os
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

In [2]:
selected_variants_file = "../data/variably_expressive/gene_variants.csv"

In [3]:
df = pd.read_csv(selected_variants_file, dtype=str)

In [4]:
def get_variant_loc_obesity_info(df):
    pivot_df = df.pivot_table(index="variant_id", values=["SYMBOL", "Mut_type", "obesity_type", "Sample"], aggfunc=lambda x: ",".join(np.unique(x)))
    pivot_df["obesity_ntypes"] = pivot_df.obesity_type.map(lambda x: len(x.split(",")))
    return pivot_df

In [5]:
vloc_df = get_variant_loc_obesity_info(df)

In [6]:
risk_only = vloc_df.loc[vloc_df.obesity_type=="risk"]
protection_only = vloc_df.loc[vloc_df.obesity_type=="protection"]
both = vloc_df.loc[vloc_df.obesity_ntypes>1]

In [7]:
def create_contingency(dfs, names):
    df = pd.concat([x.Mut_type.value_counts() for x in dfs], axis=1)
    df.columns = names
    return df

In [8]:
contingency_table = create_contingency([risk_only, protection_only, both], ["risk_only", "protection_only", "both"])

In [9]:
contingency_table

,risk_only,protection_only,both
missense,249,41,37
lof,36,5,7


In [10]:
chi2_contingency(contingency_table)

(0.541885084204419,
 0.7626603160046481,
 2,
 array([[248.52 ,  40.112,  38.368],
        [ 36.48 ,   5.888,   5.632]]))